# Registration Notebook

In [1]:
from organoid import preprocessing
from organoid import reconstruction
import numpy as np
import tifffile
from pathlib import Path
from glob import glob

c:\Users\gros\Anaconda3\envs\organoid-env\lib\site-packages\IO\IO.py:26: UserWarning: KLB library is not installed
  warnings.warn("KLB library is not installed")


pyklb library not found, klb files will not be generated


Register automatically

To register your floating image onto the reference one, you should have an idea of the transformation to apply.  From this approximative initial transformation, the algorithm will find the exact transformation to match the 2 sides.

If your image has multiple channels, one will be the reference one, registered first. The second part of the code executes the registration for the other channels, using the same transformation as computed for the reference.

In [41]:
path= rf'C:\Users\gros\Desktop\DATA\reg_test\test'
ref = tifffile.imread(rf'{path}\C1-01.tif')
# floatim = tifffile.imread(rf'{path}\2_top_dapi.tif')
print(ref.dtype)#,floatim.dtype)
# name='f32'
# tifffile.imwrite(rf'{path}\ref_{name}.tif',(ref).astype(np.float32))
# tifffile.imwrite(rf'{path}\floatim_{name}.tif',(floatim).astype(np.float32))

# print(tifffile.imread(rf'{path}\ref_{name}.tif').dtype)

uint16


In [38]:
reconstruction.register(
    path_data=Path(path) ,
    path_transformation=Path(path) / "trsf",
    path_registered_data=Path(path) / "registered",
    reference_image=f"ref_{name}.tif",
    floating_image=f"floatim_{name}.tif",
    input_voxel=[1,1,1],
    output_voxel=[1, 1, 1],
    compute_trsf=1,
    # example of transformation if the sample has been flipped between the 2 views.
    # trans1 is a translation before the rotation, trans2 is a translation after the rotation.
    # trans1=trans1,  # XYZ
    rot=[180,0,0],
    trans2=[11,-11,80],
    test_init=0,
    # other_trsf=trsf,
    trsf_type="rigid",
    depth=3,
    bbox=1,
    save_json=Path(path),
)

{'path_to_data': 'C:\\Users\\gros\\Desktop\\DATA\\reg_test\\test', 'ref_im': 'ref_f32.tif', 'flo_ims': ['floatim_f32.tif'], 'compute_trsf': 1, 'init_trsfs': [['trans', 'X', 0, 'trans', 'Y', 0, 'trans', 'Z', 0, 'rot', 'X', -180, 'rot', 'Y', 0, 'rot', 'Z', 0, 'trans', 'X', -80, 'trans', 'Y', 11, 'trans', 'Z', -11]], 'trsf_paths': ['C:\\Users\\gros\\Desktop\\DATA\\reg_test\\test\\trsf'], 'trsf_types': ['rigid'], 'ref_voxel': [1, 1, 1], 'flo_voxels': [[1, 1, 1]], 'out_voxel': [1, 1, 1], 'test_init': 0, 'apply_trsf': 1, 'out_pattern': 'C:\\Users\\gros\\Desktop\\DATA\\reg_test\\test\\registered', 'begin': 1, 'end': 1, 'bbox_out': 1, 'image_interpolation': 'linear', 'padding': 0, 'registration_depth': 3, 'ordered_init_trsfs': True}
Starting experiment
The registration will run with the following arguments:

 File format 
path_to_data             : C:\Users\gros\Desktop\DATA\reg_test\test
ref_im                   : ref_f32.tif
flo_ims                  : floatim_f32.tif
init_trsfs              

Napari visualization (you need to have napari installed)

In [68]:
# filename_ref = list_ref[0]
# filename_float = list_float[0]
channel = 'ecad'
image = tifffile.imread(Path(path) / f"{filename_ref}" / 'registered' / f"{filename_float}_{channel}.tif" )
scale = (1, 0.6,0.6)
reconstruction.check_napari(
    folder=Path(path) / f"{filename_ref}",
    reference_image=f"{filename_ref}_{channel}.tif",
    floating_image=f"{filename_float}_{channel}.tif",
    scale=scale,
)

Fuse the 2 registered sides into one array

In [35]:
for ch in channels:
    image = reconstruction.fuse_sides(
        path_registered_data=Path(path) / filename_ref / "registered",
        reference_image_reg=f"{filename_ref}_{ch}.tif",
        floating_image_reg=f"{filename_float}_{ch}.tif",
        folder_output=Path(path) / filename_ref / "fused",
        name_output=rf"fuseddata2_{ch}.tif",
        slope_coeff=25,  # slope of the weight profile : 5 corresponds to a low slope, wide fusion width and 25 to a strong slope, very thin fusion width.
    )

C:\Users\gros\Desktop\CODES\organoid\src\organoid\reconstruction\_reconstruct.py:499: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-p * (x - x0)))


Merge all the channels in one multichannel image

In [36]:
# the images should be named 'sampleid_channel.tif', eg 'fuseddata_dapi.tif', this depends on the argument "name_output" above.
reconstruction.write_hyperstacks(
    path=Path(path) / filename_ref / "fused",
    sample_id="fuseddata2",
    channels=channels,
)

Optional : Registration using landmarks

If the automatic registration is not satisfying, one option is to give more precise initial transformations to the algorithm.

For that, you need to define landmarks, using for example the Napari plugin. One landmark is a feature that you recognize in both the reference image and the floating image, that you will need to pinpoint with a marker of given label, this label has to be an integer that has the same value in both image.

Once you have at least 3 annotated landmarks (=3 labels in each image), give these landmarks to the manual_registration function, that will compute the transformation necessary to align them, and give this transformation as a starting point to the algorithm.


In [ ]:
filename_ref = list_ref[0]
filename_float = list_float[0]
scale = (1, 1, 1)
path_to_landmarks = rf'C:\Users\gros\Desktop\DATA\96h\STD_ES_40X\{filename_ref}'
reference_landmarks = tifffile.imread(
    Path(path_to_landmarks) / f"reference_landmarks.tif"
)
floating_landmarks = tifffile.imread(
    Path(path_to_landmarks) / f"floating_landmarks.tif"
)
reference_landmarks = reference_landmarks.astype(np.uint16)
floating_landmarks = floating_landmarks.astype(np.uint16)
channel = "dapi"

rot, trans1, trans2 = reconstruction.manual_registration_fct(
    reference_landmarks=reference_landmarks,
    floating_landmarks=floating_landmarks,
    scale=(1, 1, 1),
)

rot=[rot[2],rot[1],rot[0]]

reconstruction.register(
    path_data=Path(path) / filename_ref / "raw",
    path_transformation=Path(path) / filename_ref / "trsf",
    path_registered_data=Path(path) / filename_ref / "registered",
    path_to_bin=path_to_bin,
    reference_image=f"{filename_ref}_{channel}.tif",
    floating_image=f"{filename_float}_{channel}.tif",
    input_voxel=[
        1,
        1,
        1,
    ],  # if you use landmarks, better use the same voxel size in and out
    output_voxel=[1, 1, 1],
    compute_trsf=1,
    rot=rot,
    trans1=trans1,
    trans2=trans2,
    test_init=0,
    trsf_type="rigid",
    depth=3,
    bbox=1,
    save_json="",
)

det(R) < R, reflection detected!, correcting for it ...
{'path_to_bin': 'C:\\Users\\user\\Anaconda3\\envs\\organoid-env\\Library\\bin', 'path_to_data': 'C:\\Users\\gros\\Desktop\\DATA\\96h\\STD_ES_40X\\9_bottom\\raw', 'ref_im': '9_bottom_dapi.tif', 'flo_ims': ['12_top_dapi.tif'], 'compute_trsf': 1, 'init_trsfs': [['trans', 'X', 33.833333333333314, 'trans', 'Y', -6.0, 'trans', 'Z', 9.333333333333336, 'rot', 'X', -175.40241132863605, 'rot', 'Y', -3.530919075600975, 'rot', 'Z', -17.819612046606515, 'trans', 'X', -47.833333333333314, 'trans', 'Y', 30.0, 'trans', 'Z', -10.666666666666664]], 'trsf_paths': ['C:\\Users\\gros\\Desktop\\DATA\\96h\\STD_ES_40X\\9_bottom\\trsf'], 'trsf_types': ['rigid'], 'ref_voxel': [1, 1, 1], 'flo_voxels': [[1, 1, 1]], 'out_voxel': [1, 1, 1], 'test_init': 0, 'apply_trsf': 1, 'out_pattern': 'C:\\Users\\gros\\Desktop\\DATA\\96h\\STD_ES_40X\\9_bottom\\registered', 'begin': 1, 'end': 1, 'bbox_out': 1, 'image_interpolation': 'linear', 'padding': 0, 'registration_depth

C:\Users\gros\Desktop\CODES\organoid\src\organoid\reconstruction\_reconstruct.py:236: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  trsf = np.linalg.lstsq(trsf, np.identity(4))[0]


{'XResolution': 1, 'YResolution': 1, 'spacing': 1}


Debug : register the landmarks images and plot the result to see if the transformation is correct

In [ ]:
path_to_landmarks = path
path_to_bin = rf'C:\Users\gros\Anaconda3\envs\organoid-env\Library\bin'
reference_landmarks = tifffile.imread(
    Path(path_to_landmarks) / filename_ref/ f"raw/reference_landmarks.tif"
)
floating_landmarks = tifffile.imread(
    Path(path_to_landmarks) / filename_ref / f"raw/floating_landmarks.tif"
)

reference_landmarks = reference_landmarks.astype(np.uint16)
floating_landmarks = floating_landmarks.astype(np.uint16)
print(np.unique(reference_landmarks),np.shape(reference_landmarks))
print(np.unique(floating_landmarks),np.shape(floating_landmarks))
tifffile.imwrite(Path(path_to_landmarks) / filename_ref / f"raw/floating_landmarks.tif", floating_landmarks)
tifffile.imwrite(Path(path_to_landmarks) / filename_ref / f"raw/reference_landmarks.tif", reference_landmarks)
channel = "dapi"

rot, trans1, trans2 = reconstruction.manual_registration_fct(
    reference_landmarks=reference_landmarks,
    floating_landmarks=floating_landmarks,
    scale=(1, 1, 1),
)
# trans2=[0,0,0]
rot=[rot[2],rot[1],rot[0]]
reconstruction.register(
    path_data=Path(path_to_landmarks) / filename_ref / "raw",
    path_transformation=Path(path_to_landmarks) / filename_ref / "trsf",
    path_registered_data=Path(path_to_landmarks) / filename_ref / "registered",
    path_to_bin=path_to_bin,
    reference_image=f"reference_landmarks.tif",  # they have to be in 16bit to be registered
    floating_image=f"floating_landmarks.tif",
    input_voxel=[1, 1, 1],
    output_voxel=[1, 1, 1],
    compute_trsf=1,
    trans1=trans1,
    trans2=trans2,
    rot=rot,
    test_init=1,  # keep test_init=1 to see only the provided tranformation
    trsf_type="rigid",
    depth=3,
    save_json=Path(path_to_landmarks) / filename_ref,
)
print(trans1,rot,trans2)
# loading the output and add articially the center of mass of the points, to see how well they align

reference_landmarks = tifffile.imread(
    Path(path_to_landmarks) / filename_ref / f"raw/reference_landmarks.tif"
)
floating_landmarks = tifffile.imread(
    Path(path_to_landmarks) / filename_ref /f"registered/floating_landmarks.tif"
)
reference_landmarks = reconstruction.add_centermass(reference_landmarks)
floating_landmarks = reconstruction.add_centermass(floating_landmarks)

# loading the raw landmarks, before the registration
reference_landmarks_notreg = tifffile.imread(
    Path(path_to_landmarks) / filename_ref / f"raw/reference_landmarks.tif"
)
floating_landmarks_notreg = tifffile.imread(
    Path(path_to_landmarks) / filename_ref /f"raw/floating_landmarks.tif"
)
reference_landmarks_notreg = reconstruction.add_centermass(
    reference_landmarks_notreg
)
floating_landmarks_notreg = reconstruction.add_centermass(
    floating_landmarks_notreg
)

# plotting the center of the image
image_center = np.zeros_like(reference_landmarks)
z, y, x = np.array(reference_landmarks.shape) / 2
image_center[
    int(z) - 10 : int(z) + 10,
    int(y) - 10 : int(y) + 10,
    int(x) - 10 : int(x) + 10,
] = 100

reconstruction.check_napari(
    path_data=rf"{path}\{filename_ref}",
    reference_image=reference_landmarks,
    floating_image=floating_landmarks,
    additional_images=[
        reference_landmarks_notreg,
        floating_landmarks_notreg,
        image_center,
    ],
    names_additional_images=[
        "reference_landmarks_notreg",
        "floating_landmarks_notreg",
        "image_center",
    ],
    scale=scale,
    labels=True,
)

[0 1 2 3] (151, 512, 512)
[0 1 2 3] (151, 512, 512)


C:\Users\gros\Desktop\CODES\organoid\src\organoid\reconstruction\_reconstruct.py:231: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  trsf = np.linalg.lstsq(trsf, np.identity(4))[0]


{'path_to_bin': 'C:\\Users\\gros\\Anaconda3\\envs\\organoid-env\\Library\\bin', 'path_to_data': 'C:\\Users\\gros\\Desktop\\DATA\\Valentin\\48_12h_Hoechst_Ecad_Bra_Sox2\\1_bottom_norm\\raw', 'ref_im': 'reference_landmarks.tif', 'flo_ims': ['floating_landmarks.tif'], 'compute_trsf': 1, 'init_trsfs': [['trans', 'X', 32.666666666666686, 'trans', 'Y', 63.666666666666686, 'trans', 'Z', -17.5, 'rot', 'X', -167.9252700050755, 'rot', 'Y', -3.9970322766327255, 'rot', 'Z', -20.80354577451683, 'trans', 'X', -8.0, 'trans', 'Y', 88.66666666666666, 'trans', 'Z', 15.833333333333336]], 'trsf_paths': ['C:\\Users\\gros\\Desktop\\DATA\\Valentin\\48_12h_Hoechst_Ecad_Bra_Sox2\\1_bottom_norm\\trsf'], 'trsf_types': ['rigid'], 'ref_voxel': [1, 1, 1], 'flo_voxels': [[1, 1, 1]], 'out_voxel': [1, 1, 1], 'test_init': 1, 'apply_trsf': 1, 'out_pattern': 'C:\\Users\\gros\\Desktop\\DATA\\Valentin\\48_12h_Hoechst_Ecad_Bra_Sox2\\1_bottom_norm\\registered', 'begin': 1, 'end': 1, 'bbox_out': 1, 'image_interpolation': 'lin